In [53]:
# Importing dependencies
import pandas as pd
import numpy as np
import os
import pickle
import random

In [54]:
# Define the path to the CSV file
csv_file = 'hobby_dataset_rated.csv'

# Check if the CSV file exists
if os.path.isfile(csv_file):
    # Load the dataset into a pandas dataframe
    df = pd.read_csv(csv_file)

    # Check if the base_rating column already exists in the dataframe
    if 'base_rating' not in df.columns:
        # Assign a base rating of 15 to each hobby in the dataframe
        df['base_rating'] = 15

        # Save the updated dataset to a CSV file
        df.to_csv('hobby_dataset_rated.csv', index=False)
else:
    # Log missing rated file and create one using original dataset
    df = pd.read_csv('hobby_dataset_original.csv')
    df['base_rating'] = 15
    df.to_csv('hobby_dataset_rated.csv', index=False)

In [55]:
df.head(2)

,Hobby,Type of Hobby,base_rating
0,Drawing,Creative,15
1,Writing fiction,Creative,15


In [56]:
# Function to get all hobbies of a given type (Private Method)
def get_hobbies_by_type(hobby_type):
    return df[df['Type of Hobby'] == hobby_type]['Hobby'].values

In [57]:
# Function to randomly suggest a hobby of a given type
def suggest_hobby(hobby_type):
        hobbies = get_hobbies_by_type(hobby_type)
        return np.random.choice(hobbies)
   

In [58]:
# Function to update the base rating of a hobby based on user feedback
def update_rating(hobby, liked):
    if liked:
        df.loc[df['Hobby'] == hobby, 'base_rating'] += 1
        print(df.loc[df['Hobby'] == hobby]['base_rating'])
    else:
        df.loc[df['Hobby'] == hobby, 'base_rating'] -= 1
        print(df.loc[df['Hobby'] == hobby]['base_rating'])
        if df.loc[df['Hobby'] == hobby, 'base_rating'].values[0] < 0:
            df.drop(df[df['Hobby'] == hobby].index, inplace=True)

    # Save the updated dataset to a CSV file
    df.to_csv('hobby_dataset_rated.csv', index=False)

In [59]:
# Function to get recommended hobbies
def get_recommendations(n=5):
    # Calculate the weighted rating for each hobby based on its base rating
    weighted_ratings = []
    for hobby in df['Hobby'].unique():
        if hobby in df['Hobby'].values:
            weighted_rating = df.loc[df['Hobby'] == hobby, 'base_rating'].values[0]
            weighted_ratings.append((hobby, weighted_rating))

    # Sort the hobbies by weighted rating in descending order
    sorted_ratings = sorted(weighted_ratings, key=lambda x: x[1], reverse=True)

    # Return the top n hobbies with the highest weighted rating
    recommendations = []
    for hobby, rating in sorted_ratings:
        recommendations.append(hobby)
        if len(recommendations) == n:
            break
    print(recommendations)
    recommended_hobby = random.choice(recommendations)
    return recommended_hobby

In [61]:
# Testing functions - random hobby
print(suggest_hobby('Creative'))

Drawing


In [63]:
# Testing functions - update hobby liking
update_rating('Pottery', False)

4    14
Name: base_rating, dtype: int64


In [64]:
# Testing functions - get recommendation
rec_hobby = get_recommendations()
print(rec_hobby)

['Drawing', 'Writing fiction', 'Playing music', 'Photography', 'Baking']
Playing music


In [67]:
# Reset all hobbies to base rating
def reset_rating():
    df['base_rating'] = 15
    df.to_csv('hobby_dataset_rated.csv', index=False)

In [68]:
# Saves the functions as a dictionary in a pickle file because pickle only takes one parameter
functions = {'get_recommendations': get_recommendations,
             'update_rating': update_rating,
             'suggest_hobby': suggest_hobby,
             'reset_rating': reset_rating}

with open('recommendation_model.pkl', 'wb') as f:
    pickle.dump(functions, f)